<a href="https://colab.research.google.com/github/talha1503/Extractive_Text_Summarization/blob/master/Text_Rank_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import gensim
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

In [55]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pickle

path = '/content/drive/My Drive/Embeddings/glove.840B.300d.pkl'

with open(path,'rb') as f:
  embeddings = pickle.load(f)

In [0]:
from nltk.stem import WordNetLemmatizer
import re
lem = WordNetLemmatizer()

In [0]:
def clean(sentence):
  sentence = sentence.lower()
  sentence = re.sub(r'http\S+',' ',sentence)
  sentence = re.sub(r'[^a-zA-Z]',' ',sentence)
  sentence = sentence.split()
  sentence = [lem.lemmatize(word) for word in sentence if word not in stopwords.words('english')]
  sentence = ' '.join(sentence)
  return sentence

In [0]:
def average_vector(sentence):
  words = sentence.split()
  size = len(words)
  average_vector = np.zeros((size,300))
  unknown_words=[]

  for index,word in enumerate(words):
    try:  
        average_vector[index] = embeddings[word].reshape(1,-1)
    except Exception as e:
      unknown_words.append(word)
      average_vector[index] = 0

  if size!=0:
    average_vector = sum(average_vector)/size
  return average_vector,unknown_words

In [0]:
def cosine_similarity(vector_1,vector_2):
  cosine_similarity = 0
  try:
    cosine_similarity = (np.dot(vector_1,vector_2)/(np.linalg.norm(vector_1)*np.linalg.norm(vector_2)))
  except Exception as e :
    # print("Exception occured",str(e))
    pass
  return cosine_similarity

In [0]:
def find_similarity(string1,string2):
  # string1,string2 = clean(string1),clean(string2)
  vector1,unknown_words1 = average_vector(string1)
  vector2,unknown_words2 = average_vector(string2)
  similarity = cosine_similarity(vector1,vector2)
  return similarity

In [63]:
from bs4 import BeautifulSoup
import requests

subject = input("Enter the wikipedia topic to be summarised")
base_url = "https://en.wikipedia.org/wiki/"+subject
page = requests.get(base_url)

soup = BeautifulSoup(page.content,'html.parser')
paragraphs = soup.find_all('p')

content=""
for paragraph in paragraphs:
    content+=paragraph.text


Enter the wikipedia topic to be summariseddeep learning


In [0]:
sentences = sent_tokenize(content)

In [0]:
cleaned_sentences=[]
for sentence in sentences:
  cleaned_sentences.append(clean(sentence))

In [0]:
similarity_matrix = np.zeros((len(cleaned_sentences),len(cleaned_sentences)))

for i in range(0,len(cleaned_sentences)):
  for j in range(0,len(cleaned_sentences)):
    if type(find_similarity(cleaned_sentences[i],cleaned_sentences[j])) == np.float64 :
      similarity_matrix[i,j] = find_similarity(cleaned_sentences[i],cleaned_sentences[j])

In [68]:
similarity_matrix

array([[1.        , 0.59638508, 0.92437967, ..., 0.73430471, 0.53050314,
        0.        ],
       [0.59638508, 1.        , 0.51069377, ..., 0.37824843, 0.48714121,
        0.        ],
       [0.92437967, 0.51069377, 1.        , ..., 0.81988919, 0.5482137 ,
        0.        ],
       ...,
       [0.73430471, 0.37824843, 0.81988919, ..., 1.        , 0.58826502,
        0.        ],
       [0.53050314, 0.48714121, 0.5482137 , ..., 0.58826502, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [0]:
class Graph:
  
  def __init__(self,graph_dictionary):
    if not graph_dictionary:
      graph_dictionary={}
    self.graph_dictionary = graph_dictionary
  
  def vertices(self):
    return self.graph_dictionary.keys()
  
  def edges(self):
    return self.generate_edges()

  def add_vertex(self,vertex):
    if vertex not in graph_dictionary.keys():
      graph_dictionary[vertex] = []
  
  def add_edge(self,edge):
    vertex1,vertex2 = tuple(set(edge))
    if vertex1 in graph_dictionary.keys():
      graph_dictionary[vertex1].append(vertex2)
    else:
      graph_dictionary[vertex1] = [vertex2]

  def generate_edges(self):
    edges = set()
    for vertex in graph_dictionary.keys():
      for edges in graph_dictionary[vertex]:
        edges.add([vertex,edge])
    return list(edges)

In [0]:
similarity_threshold = 0.70
network_dictionary = {}

for i in range(len(cleaned_sentences)):
    network_dictionary[i] = []  

for i in range(len(cleaned_sentences)):
  for j in range(len(cleaned_sentences)):
    if similarity_matrix[i][j] > similarity_threshold:
      if j not in network_dictionary[i]:
        network_dictionary[i].append(j)
      if i not in network_dictionary[j]:
        network_dictionary[j].append(i)


In [71]:
similarity_matrix

array([[1.        , 0.59638508, 0.92437967, ..., 0.73430471, 0.53050314,
        0.        ],
       [0.59638508, 1.        , 0.51069377, ..., 0.37824843, 0.48714121,
        0.        ],
       [0.92437967, 0.51069377, 1.        , ..., 0.81988919, 0.5482137 ,
        0.        ],
       ...,
       [0.73430471, 0.37824843, 0.81988919, ..., 1.        , 0.58826502,
        0.        ],
       [0.53050314, 0.48714121, 0.5482137 , ..., 0.58826502, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [0]:
graph = Graph(network_dictionary)

In [0]:
def page_rank(graph,iterations = 50,sentences=20):
  ranks = []
  # ranks = {}
  network = graph.graph_dictionary
  current_ranks = np.squeeze(np.zeros((1,len(cleaned_sentences))))
  prev_ranks = np.array([1/len(cleaned_sentences)]*len(cleaned_sentences))
  for iteration in range(0,iterations):
    for i in range(0,len(list(network.keys()))):
      current_score = 0
      adjacent_vertices = network[list(network.keys())[i]]
      for vertex in adjacent_vertices:
          current_score += prev_ranks[vertex]/len(network[vertex])
      current_ranks[i] = current_score
    prev_ranks = current_ranks
  
  for index in range(len(cleaned_sentences)):
      # ranks[index] = prev_ranks[index]
      if prev_ranks[index]: 
        ranks.append((index,prev_ranks[index]))
  # ranks = {index:rank for index,rank in sorted(ranks.items(),key=ranks.get,reverse=True)}[:sentences]
  ranks = sorted(ranks,key = lambda x:x[1],reverse=True)[:sentences]

  return ranks

ranks = page_rank(graph,iterations=1000)

In [0]:
summary = ""
for index,rank in ranks:
  summary+=sentences[index]+" "

In [77]:
summary

'[206]\nIn further reference to the idea that artistic sensitivity might inhere within relatively low levels of the cognitive hierarchy, a published series of graphic representations of the internal states of deep (20-30 layers) neural networks attempting to discern within essentially random data the images on which they were trained[207] demonstrate a visual appeal: the original research notice received well over 1,000 comments, and was the subject of what was for a time the most frequently accessed article on The Guardian\'s[208] website. [75] However, it was discovered that replacing pre-training with large amounts of training data for straightforward backpropagation when using DNNs with large, context-dependent output layers produced error rates dramatically lower than then-state-of-the-art Gaussian mixture model (GMM)/Hidden Markov Model (HMM) and also than more-advanced generative model-based systems. [170]\nDeep learning has been shown to produce competitive results in medical a